In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [22]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.registerTempTable('green')

D:\install\spark-3.3.2-bin-hadoop3\python\pyspark\sql\dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [23]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [24]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [25]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [26]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [27]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [28]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [29]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [30]:
df_yellow_revenue.show(5)

+-------------------+----+-----------------+--------------+
|               hour|zone|           amount|number_records|
+-------------------+----+-----------------+--------------+
|2020-10-21 10:00:00| 185|            68.34|             2|
|2020-02-20 09:00:00| 142|5882.859999999997|           369|
|2021-07-27 04:00:00| 152|             29.8|             1|
|2021-01-02 21:00:00| 229|           371.29|            26|
|2020-04-01 18:00:00| 133|             8.15|             1|
+-------------------+----+-----------------+--------------+
only showing top 5 rows



In [31]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [32]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [33]:
df_join = spark.read.parquet('data/report/revenue/total')

In [35]:
df_join.show(5)

+-------------------+----+-----------------+--------------------+------------------+---------------------+
|               hour|zone|     green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+-----------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  40|           168.98|                   8| 89.97000000000001|                    5|
|2020-01-01 00:00:00|  43|           107.52|                   6| 6539.509999999994|                  390|
|2020-01-01 00:00:00|  76|           143.78|                   4|             35.51|                    2|
|2020-01-01 00:00:00|  77|75.99000000000001|                   2|            134.24|                    3|
|2020-01-01 00:00:00|  79|             null|                null|12573.810000000014|                  721|
+-------------------+----+-----------------+--------------------+------------------+---------------------+
only showing top 5 rows



In [36]:
df_zones = spark.read.parquet('zones/')

In [39]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID).drop('LocationID', 'zone')

In [40]:
df_result.show(5)

+-------------------+-----------------+--------------------+------------------+---------------------+---------+------------+
|               hour|     green_amount|green_number_records|     yellow_amount|yellow_number_records|  Borough|service_zone|
+-------------------+-----------------+--------------------+------------------+---------------------+---------+------------+
|2020-01-01 00:00:00|           168.98|                   8| 89.97000000000001|                    5| Brooklyn|   Boro Zone|
|2020-01-01 00:00:00|           107.52|                   6| 6539.509999999994|                  390|Manhattan| Yellow Zone|
|2020-01-01 00:00:00|           143.78|                   4|             35.51|                    2| Brooklyn|   Boro Zone|
|2020-01-01 00:00:00|75.99000000000001|                   2|            134.24|                    3| Brooklyn|   Boro Zone|
|2020-01-01 00:00:00|             null|                null|12573.810000000014|                  721|Manhattan| Yellow Zone|


In [41]:
df_result.write.parquet('tmp/revenue-zones', mode='overwrite')